 ╔══<i><b>&nbsp;Alai-DeepLearning&nbsp;</b></i>══════════════════════════════════╗
###  &nbsp;&nbsp; **✎&nbsp;&nbsp;Week 5. machine learning basis**
# Homework 2. Multi classification

### _Objective_
Tensorflow 을 이용해서 One Vs All 전략을 이용해 다중 classification 모델을 생성합니다.

아래 순서에 맞게 프로그램을 작성해 주세요.
1. Setosa  Vs Versicolour, Virginica 모델 구현
 - 학습시 Tensorboard 을 이용해 loss 와 accuracy 을 추적해 주세요. 
 - 학습 동안 acc 가 가장 높은 모델을 저장해 주세요.
2. Versicolour Vs Setosa, Virginica 모델 구현
  - 학습시 Tensorboard 을 이용해 loss 와 accuracy 을 추적해 주세요. 
 - 학습 동안 acc 가 가장 높은 모델을 저장해 주세요.
3. Virginica Vs Setosa, Versicolour 모델 구현
  - 학습시 Tensorboard 을 이용해 loss 와 accuracy 을 추적해 주세요. 
 - 학습 동안 acc 가 가장 높은 모델을 저장해 주세요.
4. 저장된 3개의 모델을 불러온 후 가장 높은 확률이 나오는 값을 선택합니다. <br>
가령 아래의 경우 **Versicolour** 을 선택합니다.

| class        | probabilty |
|--------------|------------|
| Setosa       | 0.7        |
| Versicolour  | 0.9        |
| Virginica    | 0.3        |

학습이 끝난후 모든 데이터를 평가한 후 accuracy 을 측정합니다.

╚═══════════════════════════════════════════════╝

In [0]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt


## Setosa  Vs Versicolour, Virginica 모델 구현 

In [0]:
import os
import numpy as np
from sklearn.datasets import load_iris
import tensorflow as tf


class IrisClassifier(object):
    def __init__(self, target_index, root_path):
        self.global_step = 0
        self.max_acc = 0
        self.graph = tf.Graph()
        self.root_path = root_path

        self.target_index = target_index
        self._data_extractor()
        self._data_normalization()
        self._build()

        with self.graph.as_default():
            self.sess = tf.Session()
            self.sess.run(tf.global_variables_initializer())

    def _data_extractor(self):
        """
        One Vs ALL 모델을 위해 Iris 데이터를 추출합니다.
        target 데이터의 라벨을 1로 하고 나머지 데이터의 라벨을 0으로 합니다.
        """
        iris = load_iris()
        self.xs_data = iris['data']
        self.ys_data = iris['target']
        self.ys_data = np.where(self.ys_data == self.target_index, [1], [0])

    def _data_normalization(self):
        self.xs_norm = (self.xs_data - self.xs_data.min(axis=0)) / \
                  (self.xs_data.max(axis=0) - self.xs_data.min(axis=0))

    def _build(self):
        with self.graph.as_default():
            self.xs = tf.placeholder(shape=[None, 4], dtype=tf.float32, name='xs')
            xs_concat = tf.concat([self.xs, tf.ones(dtype=tf.float32, shape=[tf.shape(self.xs)[0], 1])], axis=1)
            self.ys = tf.placeholder(shape=[None], dtype=tf.float32, name='ys')
            ys_res = tf.reshape(self.ys, shape=[-1, 1])
            self.lr = tf.placeholder(shape=[], dtype=tf.float32, name='lr')

            # weights
            rand_ws = tf.random_normal(shape=[4 + 1, 1], dtype=tf.float32)
            ws = tf.Variable(rand_ws, name='ws')
            layer = tf.matmul(xs_concat, ws)
            pred = 1 / (1 + tf.exp(-layer))

            # loss
            self.loss = -tf.reduce_mean(self.ys * tf.log(pred), name='loss')
            delta_ws = tf.matmul(tf.transpose(xs_concat), pred - ys_res)

            # training step
            self.step = tf.assign_sub(ws, delta_ws * self.lr, name='step')

            # Metric accruracy
            self.pred = tf.reshape(pred, shape=[-1], name='pred')
            pred_cls = tf.cast(self.pred >= 0.5, tf.float32)
            tf.where()
            self.acc = tf.reduce_mean(tf.cast(tf.equal(pred_cls, self.ys), tf.float32), name='acc')

            # add Tensorboard
            self.train_writer = tf.summary.FileWriter(os.path.join(self.root_path, 'train'))
            self.eval_writer = tf.summary.FileWriter(os.path.join(self.root_path, 'eval'))
            tf.summary.scalar(name='loss', tensor=self.loss)
            tf.summary.scalar(name='accuracy', tensor=self.acc)
            self.merged = tf.summary.merge_all()

            # Saver
            self.saver = tf.train.Saver()
            self.save_path = os.path.join(self.root_path, 'models/model')
            os.makedirs(self.save_path, exist_ok=True)

    def training(self, max_iter):
        with self.graph.as_default():
            fetches = [self.step, self.acc, self.loss, self.merged]
            feed_dict = {self.xs: self.xs_norm, self.ys: self.ys_data, self.lr: 0.01}
            acc_list = []
            loss_list = []
            for i in range(max_iter):
                _, acc_, loss_, merged_ = self.sess.run(fetches, feed_dict)
                self.global_step += 1
                self.train_writer.add_summary(merged_, self.global_step)
                acc_list.append(acc_)
                loss_list.append(loss_)
            print('Train Accuracy : {}, Loss : {}'.format(np.mean(acc_list), np.mean(loss_list)))

    def eval(self, save_flag=True):
        with self.graph.as_default():
            fetches = [self.acc, self.loss, self.merged]
            feed_dict = {self.xs: self.xs_norm, self.ys: self.ys_data}
            acc_, loss_, merged_ = self.sess.run(fetches, feed_dict)
            print('Eval Accuracy : {}, Loss : {}'.format(acc_, loss_))

            if (acc_ > self.max_acc) and save_flag:
                print('Model Saved!')
                self.saver.save(self.sess, save_path=self.save_path)

            self.eval_writer.add_summary(merged_, self.global_step)

    def restore_model(self):
        with self.graph.as_default():
            saver = tf.train.Saver()
            saver.restore(self.sess, self.save_path)

    def get_prediction(self):
        feed_dict = {self.xs: self.xs_norm}
        return self.sess.run(self.pred, feed_dict)


class OVA(object):

    def __init__(self):
        self._data_extractor()
        self._data_normalization()

        self.sento_classifier = IrisClassifier(0, 'sentosa')
        self.vesi_classifier = IrisClassifier(1, 'Versicolor')
        self.vigin_classifier = IrisClassifier(2, 'Virginica')

    def _data_extractor(self):
        """
        One Vs ALL 모델을 위해 Iris 데이터를 추출합니다.
        target 데이터의 라벨을 1로 하고 나머지 데이터의 라벨을 0으로 합니다.
        """
        iris = load_iris()
        self.xs_data = iris['data']
        self.ys_data = iris['target']

    def _data_normalization(self):
        self.xs_norm = (self.xs_data - self.xs_data.min(axis=0)) / \
                  (self.xs_data.max(axis=0) - self.xs_data.min(axis=0))

    def training(self):
        self.sento_classifier.training(max_iter=1000)
        self.sento_classifier.eval()

        self.vesi_classifier.training(max_iter=1000)
        self.vesi_classifier.eval()

        self.vigin_classifier.training(max_iter=1000)
        self.vigin_classifier.eval()

    def restore_models(self):
        classifiers = [self.sento_classifier, self.vesi_classifier, self.vigin_classifier]
        for classifier in classifiers:
            with classifier.graph.as_default():
                saver = tf.train.Saver()
                saver.restore(classifier.sess, classifier.save_path)

    def evaluation(self):
        predictions = [self.sento_classifier.get_prediction(), self.vesi_classifier.get_prediction(),
                       self.vigin_classifier.get_prediction()]
        pred_cls = np.argmax(np.stack(predictions, axis=-1), axis=1)
        acc = np.mean(self.ys_data == pred_cls)
        return acc

if __name__ == '__main__':

    ova = OVA()
    ova.training()
    ova.evaluation()
    ova.training()
    ova.evaluation()



INFO:tensorflow:Restoring parameters from sentosa/models/model
INFO:tensorflow:Restoring parameters from Versicolor/models/model
INFO:tensorflow:Restoring parameters from Virginica/models/model
0.9466666666666667


In [0]:
# 다시 학습하지 않고 저장된 모델을 불러와 평가할때 사용됩니다. 
# 저장된 모델 불러와서 평가.
ova = OVA()
ova.restore_models
ova.evaluation()


Instructions for updating:
Colocations handled automatically by placer.
Train Accuracy : 0.9995866417884827, Loss : 1.3736305236816406
Eval Accuracy : 1.0, Loss : 1.6921262741088867
Model Saved!
Train Accuracy : 0.7206200361251831, Loss : 0.4587215483188629
Eval Accuracy : 0.7266666889190674, Loss : 0.4884854257106781
Model Saved!
Train Accuracy : 0.9667065143585205, Loss : 1.2461198568344116
Eval Accuracy : 0.9733333587646484, Loss : 1.7310360670089722
Model Saved!
